In [54]:
import numpy as np
import pandas as pd
import math
from scipy import stats
from statistics import mean
import requests
import xlsxwriter

In [19]:
stocks = pd.read_csv('sp500_stocks.csv')
from api_key import IEX_CLOUD_API_TOKEN

In [20]:
stocks

,Symbol
0,AAPL
1,MSFT
2,GOOG
3,GOOGL
4,AMZN
...,...
498,VNO
499,PENN
500,ALK
501,PVH


In [21]:
def batch(lst,size):
    symbol_group = []
    for i in range(0,len(lst),size):
        symbol_group.append(lst[i:i+size])       
    return symbol_group
        

In [26]:
symbol_group = batch(stocks['Symbol'],100)
symbol_strings = []
for i in range(len(symbol_group)):
    symbol_strings.append(','.join(symbol_group[i]))

In [50]:
column_names = ['Symbol','Price','Price to Earnings Ratio','PE Percentile','Price to Book Ratio','PB Percentile',
                'Price to Sales Ratio', 'PS Percentile','EV/EBITDA', 'EV/EBITDA Percentile',
                'EV/GP','EV/GP Percentile','Score', 'No.of Shares to Buy']
df = pd.DataFrame(columns = column_names)
for symbols in symbol_strings:
    batch_api = f'https://sandbox.iexapis.com/stable//stock/market/batch?symbols={symbols}&types=advanced-stats,price&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api).json()
#     print(data)
    for s in symbols.split(','):
        ev = data[s]['advanced-stats']['enterpriseValue']
        ebitda = data[s]['advanced-stats']['EBITDA']
        if ev and ebitda:
            ev_to_ebitda = ev/ebitda
        else:
            ev_to_ebitda = np.NaN
        if ev and gp:
            ev_to_gp = ev/gp
        else:
            ev_to_gp = np.NaN
        gp = data[s]['advanced-stats']['grossProfit']
        df = df.append(pd.Series([s,data[s]['price'],data[s]['advanced-stats']['peRatio'],'N/A',
                                  data[s]['advanced-stats']['priceToBook'],'N/A',
                                  data[s]['advanced-stats']['priceToSales'],'N/A',
                                  ev_to_ebitda,'N/A',
                                  ev_to_gp,'N/A','N/A','N/A'],index = column_names),ignore_index=True)


In [51]:
cnames = ['Price','Price to Earnings Ratio','Price to Book Ratio', 'Price to Sales Ratio','EV/EBITDA', 'EV/GP']
for col in cnames:
    df[col].fillna(df[col].mean(),inplace = True)

In [55]:
percentile = {
    'Price to Earnings Ratio':'PE Percentile',
    'Price to Book Ratio':'PB Percentile',
    'Price to Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile'
}
for row in df.index:
    values = []
    for key in percentile:
        df.loc[row,percentile[key]] = stats.percentileofscore(df[key],df.loc[row,key])
        values.append(df.loc[row,percentile[key]])
    df.loc[row,'Score'] = mean(values)

df
        

,Symbol,Price,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Score,No.of Shares to Buy
0,AAPL,162.61,26.015089,60.039761,38.930000,97.017893,6.90000,83.399602,19.333893,69.582505,4311.376903,100.0,82.007952,N/A
1,MSFT,287.31,29.196248,68.190855,13.300000,90.457256,10.83000,92.047714,21.390007,74.751491,11.954539,75.944334,80.27833,N/A
2,GOOG,122.01,26.138930,61.332008,3.960788,58.151093,4.17451,67.296223,36.590149,94.73161,17.046603,89.761431,74.254473,N/A
3,GOOGL,119.63,1.044209,3.578529,0.311000,6.560636,0.29740,1.39165,-0.436877,1.39165,17.046603,89.761431,20.536779,N/A
4,AMZN,141.58,60.521221,93.439364,9.360000,82.803181,2.70000,45.924453,24.004569,82.50497,8.399210,62.425447,73.419483,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,VNO,31.34,48.701409,89.860835,1.190000,12.524851,3.64000,59.44334,14.926076,53.87674,4.244198,29.224652,48.986083,N/A
499,PENN,35.90,15.070439,31.212724,1.470000,18.489066,0.95330,13.518887,6.658784,13.916501,6.370316,44.930417,24.413519,N/A
500,ALK,45.22,12.319457,23.459245,1.580000,20.775348,0.80000,10.337972,8.566614,24.254473,1.766211,9.145129,17.594433,N/A
501,PVH,62.70,4.268980,4.771372,0.797300,8.548708,0.45190,3.578529,4.503777,5.765408,0.779292,2.186879,4.970179,N/A


Choosing the least value stocks 

In [56]:
df.sort_values('Score',inplace = True)

In [59]:
df = df[0:30]
df

,Symbol,Price,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Score,No.of Shares to Buy
0,PVH,62.70,4.268980,4.771372,0.7973,8.548708,0.4519,3.578529,4.503777,5.765408,0.779292,2.186879,4.970179,N/A
1,PRU,104.69,7.899891,11.928429,0.8672,8.946322,0.5589,5.964215,5.120149,8.151093,1.859083,9.940358,8.986083,N/A
2,ALL,119.67,9.202065,15.506958,1.5000,18.986083,0.6466,7.554672,3.008248,2.7833,0.472189,0.795229,9.125249,N/A
3,LNC,51.78,6.752093,9.940358,0.6183,7.554672,0.4563,3.777336,5.179864,8.946322,3.014077,18.88668,9.821074,N/A
4,HIG,64.51,8.356466,13.518887,1.3600,16.202783,0.9488,13.32008,4.319103,5.367793,1.162108,4.771372,10.636183,N/A
5,DXC,32.80,10.604271,19.681909,1.4900,18.687873,0.4615,4.17495,4.456303,5.5666,1.441645,6.163022,10.854871,N/A
6,PARA,24.47,3.931837,4.17495,0.7330,7.753479,0.5619,6.163022,7.631545,20.477137,2.931589,17.693837,11.252485,N/A
7,RE,265.85,8.062907,12.326044,1.1100,11.729622,0.8722,12.127237,6.986796,16.699801,0.936899,3.379722,11.252485,N/A
8,COF,113.61,3.985337,4.373757,0.7827,8.151093,1.3100,19.980119,2.955728,2.385686,3.326612,22.266402,11.431412,N/A
9,AIZ,182.27,7.140082,10.536779,1.8700,26.341948,0.9176,12.723658,2.545419,1.988072,1.571843,7.157058,11.749503,N/A


In [60]:
def invest():
    flag = True
    while(flag):
        amount = input('Enter your investment in dollars\n')
        try:
            val = float(amount)
        except ValueError:
            print('Investment accepts only numbers try again\n')
        else:
            flag = False
    return float(amount)

In [61]:
investment = invest()

Enter your investment in dollars
300000


In [62]:
investment_size = investment/len(df.index)
investment_size

10000.0

In [63]:
for row in df.index:
    df.loc[row,'No.of Shares to Buy'] = math.floor(investment_size/df.loc[row,'Price'])

In [64]:
df

,Symbol,Price,Price to Earnings Ratio,PE Percentile,Price to Book Ratio,PB Percentile,Price to Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Score,No.of Shares to Buy
0,PVH,62.70,4.268980,4.771372,0.7973,8.548708,0.4519,3.578529,4.503777,5.765408,0.779292,2.186879,4.970179,159
1,PRU,104.69,7.899891,11.928429,0.8672,8.946322,0.5589,5.964215,5.120149,8.151093,1.859083,9.940358,8.986083,95
2,ALL,119.67,9.202065,15.506958,1.5000,18.986083,0.6466,7.554672,3.008248,2.7833,0.472189,0.795229,9.125249,83
3,LNC,51.78,6.752093,9.940358,0.6183,7.554672,0.4563,3.777336,5.179864,8.946322,3.014077,18.88668,9.821074,193
4,HIG,64.51,8.356466,13.518887,1.3600,16.202783,0.9488,13.32008,4.319103,5.367793,1.162108,4.771372,10.636183,155
5,DXC,32.80,10.604271,19.681909,1.4900,18.687873,0.4615,4.17495,4.456303,5.5666,1.441645,6.163022,10.854871,304
6,PARA,24.47,3.931837,4.17495,0.7330,7.753479,0.5619,6.163022,7.631545,20.477137,2.931589,17.693837,11.252485,408
7,RE,265.85,8.062907,12.326044,1.1100,11.729622,0.8722,12.127237,6.986796,16.699801,0.936899,3.379722,11.252485,37
8,COF,113.61,3.985337,4.373757,0.7827,8.151093,1.3100,19.980119,2.955728,2.385686,3.326612,22.266402,11.431412,88
9,AIZ,182.27,7.140082,10.536779,1.8700,26.341948,0.9176,12.723658,2.545419,1.988072,1.571843,7.157058,11.749503,54
